# https://github.com/microsoft/promptflow/blob/user/singankit/pf-evals-bug-bash/src/promptflow-evals/samples/bug-bash/instructions.md

In [ ]:
import os
import json

from promptflow.core import AzureOpenAIModelConfiguration

# Initialize Azure OpenAI Connection
model_config = AzureOpenAIModelConfiguration(
        azure_deployment="gpt-4",
        api_version=os.environ["AZURE_OPENAI_API_VERSION"],
        azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"]
    )

In [ ]:
import pandas as pd

data_path = "../data/data.jsonl"

df = pd.read_json(data_path, lines=True)
df.head()

In [ ]:
import sys
sys.path.append('../contoso_chat')  # Replace '/path/to/contoso_chat' with the actual path to the 'contoso_chat' folder

from chat_request import get_response
from promptflow.evals.evaluators import RelevanceEvaluator, GroundednessEvaluator, FluencyEvaluator, CoherenceEvaluator

relevance_evaluator = RelevanceEvaluator(model_config)
groundedness_evaluator = GroundednessEvaluator(model_config)
fluency_evaluator = FluencyEvaluator(model_config)
coherence_evaluator = CoherenceEvaluator(model_config)

In [ ]:
# create unique id for each run with date and time
from datetime import datetime
run_id = datetime.now().strftime("%Y%m%d%H%M%S")
run_id = f"{run_id}_chat_evaluation_sdk"    
print(run_id)

In [ ]:
from promptflow.evals.evaluate import evaluate

result_eval = evaluate(
    evaluation_name=run_id,
    data="../data/data.jsonl",
    target=get_response,
    evaluators={
        #"violence": violence_eval,
        "relevance": relevance_evaluator,
        "fluency": fluency_evaluator,
        "coherence": coherence_evaluator,
        "groundedness": groundedness_evaluator,
    },
    # column mapping    return {"question": question, "answer": result, "context": context}
    evaluator_config={
        "defaultS": {
            "question": "${data.question}",
            "answer": "${target.answer}",
            "context": "${target.context}",
        },
    },
)

In [ ]:
eval_result = pd.DataFrame(result_eval["rows"])

In [ ]:
eval_result.head()

In [ ]:
#save evaluation results to a JSONL file
eval_result.to_json('eval_result.jsonl', orient='records', lines=True)